# 🧠 Tutorial 2: Google ADK - ¡Controla tus LLMs!

## LiteLLM, Parámetros y Output Estructurado

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/)

### 📋 ¿Qué aprenderás en este tutorial?

1. **🔄 Flexibilidad de Modelos con LiteLLM**
   - Usar Claude, GPT, Llama y otros modelos en ADK
   - Configuración de múltiples proveedores

2. **⚙️ Ajustar el Comportamiento del LLM**
   - Parámetros clave: temperature, top_p, max_tokens
   - Casos de uso para diferentes configuraciones

3. **📊 Output Estructurado con Pydantic**
   - Definir esquemas de datos
   - Obtener respuestas JSON predecibles

4. **🚀 Ejemplos Prácticos y Avanzados**
   - Comparación entre modelos
   - Extracción de información compleja

---

### 🎯 Objetivo del Tutorial

Después de completar este tutorial, serás capaz de:
- Integrar cualquier LLM en tus agentes ADK
- Ajustar finamente el comportamiento de los modelos
- Obtener respuestas estructuradas y validadas

**Requisitos previos:**
- Haber completado el Tutorial 1 de ADK
- API Keys de los modelos que quieras usar

## 🔧 Configuración Inicial

### Instalación de Dependencias

In [ ]:
# Instalar Google ADK con soporte para LiteLLM
print("📦 Instalando Google ADK con LiteLLM...")
!pip install -q google-adk==1.4.2
!pip install -q litellm==1.73.0
!pip install -qU python-dotenv pydantic

print("\n✅ Instalación completada!")

# Verificar versiones
import sys
print(f"\n🐍 Python: {sys.version.split()[0]}")
!pip show google-adk litellm pydantic | grep -E "Name:|Version:"

### Configuración de API Keys

Para este tutorial, necesitarás al menos una API Key. Puedes obtenerlas de:
- **Google AI Studio**: [https://aistudio.google.com/apikey](https://aistudio.google.com/apikey)
- **Anthropic (Claude)**: [https://console.anthropic.com/](https://console.anthropic.com/)
- **OpenAI**: [https://platform.openai.com/](https://platform.openai.com/)

#### Opcion 1: Ingresalas directamente

In [1]:
import os
from getpass import getpass

print("🔑 Configuración de API Keys\n")
print("Ingresa las API Keys que tengas disponibles (presiona Enter para omitir):\n")

# Google API Key (requerida para ejemplos base)
if 'GOOGLE_API_KEY' not in os.environ:
    google_key = getpass("Google API Key (requerida): ")
    if google_key:
        os.environ['GOOGLE_API_KEY'] = google_key
        os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'

# Anthropic API Key (opcional)
if 'ANTHROPIC_API_KEY' not in os.environ:
    anthropic_key = getpass("Anthropic API Key (opcional): ")
    if anthropic_key:
        os.environ['ANTHROPIC_API_KEY'] = anthropic_key

# OpenAI API Key (opcional)
if 'OPENAI_API_KEY' not in os.environ:
    openai_key = getpass("OpenAI API Key (opcional): ")
    if openai_key:
        os.environ['OPENAI_API_KEY'] = openai_key

# Verificar configuración
print("\n📋 Estado de las API Keys:")
print(f"   Google: {'✅' if os.environ.get('GOOGLE_API_KEY') else '❌'}")
print(f"   Anthropic: {'✅' if os.environ.get('ANTHROPIC_API_KEY') else '❌'}")
print(f"   OpenAI: {'✅' if os.environ.get('OPENAI_API_KEY') else '❌'}")

🔑 Configuración de API Keys

Ingresa las API Keys que tengas disponibles (presiona Enter para omitir):


📋 Estado de las API Keys:
   Google: ✅
   Anthropic: ✅
   OpenAI: ✅


#### Opcion 2: Usar dotenv

In [1]:
from dotenv import load_dotenv
# Cargar variables de entorno desde .env si existe
load_dotenv(override=True)

True

## 🔄 Parte 1: Flexibilidad de Modelos con LiteLLM

### ¿Qué es LiteLLM?

LiteLLM es una biblioteca que proporciona una interfaz unificada para más de 100 modelos de lenguaje diferentes. Actúa como un "traductor universal" entre ADK y los diversos proveedores de LLMs.

### Ventajas de usar LiteLLM:

- 🧪 **Experimentación fácil**: Prueba diferentes modelos sin cambiar tu código
- 💰 **Optimización de costos**: Elige el modelo más económico para cada tarea
- 🔓 **Sin vendor lock-in**: Libertad para cambiar de proveedor
- 🎯 **Modelos especializados**: Usa el mejor modelo para cada caso

### Crear Agentes con Diferentes Modelos

In [7]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.agents.llm_agent import LlmAgent
from google.genai import types


async def call_agent_async(query: str, runner, user_id, session_id):
    """Envía una consulta al agente e imprime la respuesta final."""
    print(f"\n>>> Consulta del usuario: {query}")

    # Prepara el mensaje del usuario en el formato de ADK
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "El agente no produjo una respuesta final." # Valor por defecto

    # Concepto clave: run_async ejecuta la lógica del agente y genera eventos.
    # Iteramos a través de los eventos para encontrar la respuesta final.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # Puedes descomentar la línea de abajo para ver *todos* los eventos durante la ejecución
        # print(f"  [Evento] Autor: {event.author}, Tipo: {type(event).__name__}, Final: {event.is_final_response()}, Contenido: {event.content}")

        # Concepto clave: is_final_response() marca el mensaje que concluye el turno.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Se asume que la respuesta de texto está en la primera parte
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Maneja posibles errores/escalamientos
                final_response_text = f"El agente escaló: {event.error_message or 'Sin mensaje específico.'}"
            # Agrega más validaciones aquí si es necesario (por ejemplo, códigos de error específicos)
            break # Deja de procesar eventos una vez encontrada la respuesta final

    print(f"<<< Respuesta del agente: {final_response_text}")


### Primero Gemimi de Google

In [8]:
MODEL_GEMINI = "gemini-2.5-flash"

# Example: Defining the basic Agent
refranes_agent = LlmAgent(
    model=MODEL_GEMINI,
    name="refranes_agent",
    description="completa los refranes que el usuario empieza"
)

In [9]:
session_service = InMemorySessionService()

APP_NAME = "test_gemini"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
# Runner: This is the main component that manages the interaction with the agent.
runner_gemini = Runner(agent=refranes_agent,app_name=APP_NAME,session_service=session_service)


In [10]:
await call_agent_async("A caballo regalado...",
                           runner=runner_gemini,
                           user_id=USER_ID,
                           session_id=SESSION_ID)


>>> Consulta del usuario: A caballo regalado...
<<< Respuesta del agente: A caballo regalado no se le mira el diente.


## Vamos con OpenAI

In [11]:
from google.adk.models.lite_llm import LiteLlm
openai_model = LiteLlm("openai/gpt-4.1")

In [12]:
refranes_agent_openai = LlmAgent(
    model=openai_model,
    name="refranes_agent",
    description="completa los refranes que el usuario empieza",
)

APP_NAME = "test_openai"
USER_ID = "user_2"
SESSION_ID = "session_002" # Using a fixed ID for simplicity

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_openai = Runner(agent=refranes_agent_openai,app_name=APP_NAME,session_service=session_service)

In [13]:
await call_agent_async("Camaron que se duerme...",
                           runner=runner_openai,
                           user_id=USER_ID,
                           session_id=SESSION_ID)


>>> Consulta del usuario: Camaron que se duerme...
<<< Respuesta del agente: ...se lo lleva la corriente.


## Probemos Anthropic

In [14]:
anthropic_model = LiteLlm("anthropic/claude-3-7-sonnet-20250219")
refranes_agent_claude = LlmAgent(
    model=anthropic_model,
    name="refranes_agent",
    description="completa los refranes que el usuario empieza",
)

APP_NAME = "test_claude"
USER_ID = "user_3"
SESSION_ID = "session_003" # Using a fixed ID for simplicity

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_openai = Runner(agent=refranes_agent_claude,app_name=APP_NAME,session_service=session_service)

In [15]:
await call_agent_async("Arbol que nace torcido...",
                           runner=runner_openai,
                           user_id=USER_ID,
                           session_id=SESSION_ID)


>>> Consulta del usuario: Arbol que nace torcido...
<<< Respuesta del agente: Árbol que nace torcido, nunca su tronco endereza.

Este refrán sugiere que cuando algo o alguien desarrolla ciertos hábitos o características desde el principio, es muy difícil cambiarlos más adelante. Se refiere a la importancia de la educación y formación temprana en el desarrollo de una persona.

¿Te gustaría que te complete algún otro refrán?


## Incluso con Azure - OpenAI

In [16]:
azure_model = LiteLlm("azure/gpt-4o")
refranes_agent_azure = LlmAgent(
    model=azure_model,
    name="refranes_agent",
    description="completa los refranes que el usuario empieza",
)

APP_NAME = "test_azure"
USER_ID = "user_4"
SESSION_ID = "session_004" # Using a fixed ID for simplicity


session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_openai = Runner(agent=refranes_agent_azure,app_name=APP_NAME,session_service=session_service)

In [17]:
await call_agent_async("Escoba nueva...",
                           runner=runner_openai,
                           user_id=USER_ID,
                           session_id=SESSION_ID)


>>> Consulta del usuario: Escoba nueva...
<<< Respuesta del agente: ¡barre bien!


## Incluso modelos locales

In [18]:
ollama_model = LiteLlm("ollama/gemma3:4b")
refranes_agent_ollama = LlmAgent(
    model=ollama_model,
    name="refranes_agent",
    description="completa los refranes que el usuario empieza",
)

APP_NAME = "test_ollama"
USER_ID = "user_5"
SESSION_ID = "session_005" # Using a fixed ID for simplicity


session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_openai = Runner(agent=refranes_agent_ollama,app_name=APP_NAME,session_service=session_service)

In [19]:
await call_agent_async("mas vale pajaro en mano...",
                        runner=runner_openai,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


>>> Consulta del usuario: mas vale pajaro en mano...
<<< Respuesta del agente: mas vale pájaro en mano, que mil volando.



## ⚙️ Parte 2: Ajustando el Comportamiento con Parámetros

### Parámetros Clave de los LLMs

Los parámetros permiten controlar cómo los modelos generan texto:

1. **🌡️ Temperature (0.0 - 2.0)**
   - Baja (0.0-0.3): Respuestas deterministas y conservadoras
   - Media (0.4-0.7): Balance entre consistencia y creatividad
   - Alta (0.8-2.0): Respuestas creativas y diversas

2. **🎯 Top-p (0.0 - 1.0)**
   - Controla el "nucleus sampling"
   - 0.9 = considera tokens que suman 90% de probabilidad
   - Alternativa a temperature (usar uno u otro)

3. **📏 Max Output Tokens**
   - Limita la longitud de la respuesta
   - Útil para controlar costos y concisión

In [20]:
# Agente Creativo (alta temperatura)
agente_creativo = LlmAgent(
    name="AgenteCreativo",
    model=azure_model,
    description="Agente configurado para máxima creatividad",
    generate_content_config=types.GenerateContentConfig(
        temperature= 1.5,          # Alta creatividad
        max_output_tokens = 1000,    # Respuestas moderadas
        top_k= 40                  # Vocabulario amplio
     ),
    instruction=(
        "Eres un escritor creativo e imaginativo."
        "Genera ideas originales y sorprendentes."
        "Usa metáforas, analogías y lenguaje colorido."
    )
)

# Agente Técnico (baja temperatura)
agente_tecnico = LlmAgent(
    name="AgenteTecnico",
    model=azure_model,
    description="Agente configurado para precisión técnica",
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.1,          # Muy determinista
        max_output_tokens= 150    # Respuestas concisas
    ),
    instruction=(
        "Eres un experto técnico preciso y factual."
        "Proporciona información exacta y verificable."
        "Evita especulaciones y cíñete a los hechos."
    )
)

# Agente Balanceado (configuración media)
agente_balanceado = LlmAgent(
    name="AgenteBalanceado",
    model=azure_model,
    description="Agente con configuración equilibrada",
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.7,          # Balance
        max_output_tokens= 300    # Flexibilidad en longitud
    ),
    instruction=(
        "Eres un asistente versátil y adaptable."
        "Proporciona respuestas útiles y bien estructuradas."
        "Adapta tu estilo según el contexto."
    )
)

# Agente Ultra-Conciso (tokens limitados)
agente_conciso = LlmAgent(
    name="AgenteConciso",
    model=azure_model,
    description="Agente de respuestas ultra-breves",
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.3,
        max_output_tokens= 50     # Muy limitado
    ),
    instruction=(
        "Responde de forma extremadamente concisa."
        "Máximo 2-3 frases por respuesta."
        "Ve directo al punto, sin rodeos."
    )
)

print("🎛️ Agentes con diferentes parámetros creados:")
print(" • AgenteCreativo (temp=1.5)")
print(" • AgenteTecnico (temp=0.1)")
print(" • AgenteBalanceado (temp=0.7)")
print(" • AgenteConciso (max_tokens=50)")

🎛️ Agentes con diferentes parámetros creados:
 • AgenteCreativo (temp=1.5)
 • AgenteTecnico (temp=0.1)
 • AgenteBalanceado (temp=0.7)
 • AgenteConciso (max_tokens=50)


### 🧪 Demostración: Efecto de los Parámetros

In [21]:
APP_NAME = "test_creative_agent"
USER_ID = "user_6"
SESSION_ID = "session_006" # Using a fixed ID for simplicity

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_creativo = Runner(agent=agente_creativo,app_name=APP_NAME,session_service=session_service)

await call_agent_async("Escribe un poema sobre la luna llena",
                        runner=runner_creativo,
                        user_id=USER_ID,
                        session_id=SESSION_ID)




>>> Consulta del usuario: Escribe un poema sobre la luna llena
<<< Respuesta del agente: **Bailarina de Plata**  

Oh, luna llena, diadema del cielo,  
caricia brillante en el manto negro,  
bailarina etérea de luz y misterio,  
te deslizas callada en un vasto silencio.  

Eres el ojo abierto de la noche fría,  
un espejo partido en la magia tardía,  
tu rostro se cuelga cual fruta prohibida,  
mordida ya, solo por sueños y envidia.  

Juegas con sombras, dibujas catedrales,  
sobre las montañas, los ríos, los valles,  
y entre los sauces, como augusta reina,  
coronas el mundo con tu calma plena.  

Beso de plata sobre el lomo del mar,  
reflejo tembloroso que ansía alcanzar  
esa dama esquiva que vive en el cielo,  
surcando las horas en un suave vuelo.  

Los lobos aúllan tu nombre secreto,  
y mil corazones laten un soneto.  
Eres poema, eterna y desnuda,  
musa brillante de la noche muda.  

Oh luna llena, fanal de ensueño,  
te inclinas paciente al nocturno dueño.  
Pero esta no

In [22]:
APP_NAME = "test_conciso_agent"
USER_ID = "user_6"
SESSION_ID = "session_006" # Using a fixed ID for simplicity


session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_consiso = Runner(agent=agente_conciso,app_name=APP_NAME,session_service=session_service)

await call_agent_async("Escribe un poema sobre la luna llena",
                        runner=runner_consiso,
                        user_id=USER_ID,
                        session_id=SESSION_ID)




>>> Consulta del usuario: Escribe un poema sobre la luna llena
<<< Respuesta del agente: Luna redonda,  
farol en la noche,  
calma y asombro.


### 📊 Guía de Uso de Parámetros

Recomendaciones según el caso de uso:

In [23]:
# Crear una guía visual de parámetros
import pandas as pd

# Crear tabla de recomendaciones
guia_parametros = pd.DataFrame([
    {"Caso de Uso": "Documentación Técnica", 
     "Temperature": "0.1-0.3", 
     "Max Tokens": "500-1000", 
     "Top-p": "0.9-0.95",
     "Razón": "Precisión y consistencia"},
    
    {"Caso de Uso": "Escritura Creativa", 
     "Temperature": "0.8-1.5", 
     "Max Tokens": "200-500", 
     "Top-p": "0.95-1.0",
     "Razón": "Originalidad y variedad"},
    
    {"Caso de Uso": "Chatbot de Servicio", 
     "Temperature": "0.5-0.7", 
     "Max Tokens": "100-200", 
     "Top-p": "0.9",
     "Razón": "Balance y eficiencia"},
    
    {"Caso de Uso": "Análisis de Datos", 
     "Temperature": "0.1-0.2", 
     "Max Tokens": "300-500", 
     "Top-p": "0.95",
     "Razón": "Exactitud en resultados"},
    
    {"Caso de Uso": "Brainstorming", 
     "Temperature": "1.0-1.8", 
     "Max Tokens": "150-300", 
     "Top-p": "0.95-1.0",
     "Razón": "Máxima diversidad de ideas"}
])

print("📊 GUÍA DE PARÁMETROS SEGÚN CASO DE USO\n")
print(guia_parametros.to_string(index=False))

print("\n\n⚠️ Notas importantes:")
print("   • Estos son rangos sugeridos, experimenta según tus necesidades")
print("   • No uses temperature y top-p simultáneamente en valores extremos")
print("   • El costo aumenta con max_tokens, úsalo con prudencia")
print("   • Algunos modelos tienen límites específicos, consulta la documentación")

📊 GUÍA DE PARÁMETROS SEGÚN CASO DE USO

          Caso de Uso Temperature Max Tokens    Top-p                      Razón
Documentación Técnica     0.1-0.3   500-1000 0.9-0.95   Precisión y consistencia
   Escritura Creativa     0.8-1.5    200-500 0.95-1.0    Originalidad y variedad
  Chatbot de Servicio     0.5-0.7    100-200      0.9       Balance y eficiencia
    Análisis de Datos     0.1-0.2    300-500     0.95    Exactitud en resultados
        Brainstorming     1.0-1.8    150-300 0.95-1.0 Máxima diversidad de ideas


⚠️ Notas importantes:
   • Estos son rangos sugeridos, experimenta según tus necesidades
   • No uses temperature y top-p simultáneamente en valores extremos
   • El costo aumenta con max_tokens, úsalo con prudencia
   • Algunos modelos tienen límites específicos, consulta la documentación


## 📊 Parte 3: Output Estructurado con Pydantic

### ¿Por qué Output Estructurado?

El output estructurado es crucial cuando necesitas:
- 🔧 Integrar respuestas de IA con otros sistemas
- 💾 Almacenar datos en bases de datos
- 🎯 Garantizar formato consistente
- ✅ Validar la información extraída

### Pydantic: La Solución

Pydantic permite definir esquemas de datos con:
- Type hints de Python
- Validación automática
- Serialización JSON
- Documentación clara para el LLM

In [24]:
# Importar Pydantic y crear modelos de datos
from pydantic import BaseModel, Field
from typing import List, Optional

print("📚 Creando modelos Pydantic para output estructurado...\n")

# Modelo 1: Información de Producto
class InformacionProducto(BaseModel):
    """Esquema para extraer información de productos"""
    nombre: str = Field(description="Nombre completo del producto")
    marca: Optional[str] = Field(None, description="Marca del producto")
    precio: Optional[float] = Field(None, description="Precio en USD")
    caracteristicas: List[str] = Field(
        default_factory=list,
        description="Lista de características principales"
    )
    disponible: bool = Field(True, description="Si está disponible")
    categoria: Optional[str] = Field(None, description="Categoría del producto")

# Modelo 2: Análisis de Sentimiento
class AnalisisSentimiento(BaseModel):
    """Esquema para análisis de sentimiento de texto"""
    sentimiento: str = Field(
        description="Sentimiento general: positivo, negativo o neutral"
    )
    confianza: float = Field(
        description="Nivel de confianza del análisis (0.0 a 1.0)",
        ge=0.0, le=1.0
    )
    emociones: List[str] = Field(
        default_factory=list,
        description="Emociones detectadas en el texto"
    )
    aspectos_positivos: List[str] = Field(
        default_factory=list,
        description="Aspectos positivos mencionados"
    )
    aspectos_negativos: List[str] = Field(
        default_factory=list,
        description="Aspectos negativos mencionados"
    )

# Modelo 3: Extracción de Eventos
class Evento(BaseModel):
    """Información sobre un evento"""
    titulo: str = Field(description="Título del evento")
    fecha: Optional[str] = Field(None, description="Fecha del evento (YYYY-MM-DD)")
    hora: Optional[str] = Field(None, description="Hora del evento (HH:MM)")
    ubicacion: Optional[str] = Field(None, description="Ubicación del evento")
    participantes: List[str] = Field(
        default_factory=list,
        description="Lista de participantes"
    )
    descripcion: Optional[str] = Field(None, description="Descripción del evento")

class ListaEventos(BaseModel):
    """Lista de eventos extraídos"""
    eventos: List[Evento] = Field(
        default_factory=list,
        description="Lista de todos los eventos encontrados"
    )
    total_eventos: int = Field(
        description="Número total de eventos encontrados"
    )

print("✅ Modelos Pydantic creados:")
print("   1. InformacionProducto - Para extraer datos de productos")
print("   2. AnalisisSentimiento - Para análisis de opiniones")
print("   3. Evento/ListaEventos - Para extraer información de eventos")

# Ejemplo de uso
print("\n📋 Ejemplo de esquema (InformacionProducto):")
print(InformacionProducto.model_json_schema()['properties'])

📚 Creando modelos Pydantic para output estructurado...

✅ Modelos Pydantic creados:
   1. InformacionProducto - Para extraer datos de productos
   2. AnalisisSentimiento - Para análisis de opiniones
   3. Evento/ListaEventos - Para extraer información de eventos

📋 Ejemplo de esquema (InformacionProducto):
{'nombre': {'description': 'Nombre completo del producto', 'title': 'Nombre', 'type': 'string'}, 'marca': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Marca del producto', 'title': 'Marca'}, 'precio': {'anyOf': [{'type': 'number'}, {'type': 'null'}], 'default': None, 'description': 'Precio en USD', 'title': 'Precio'}, 'caracteristicas': {'description': 'Lista de características principales', 'items': {'type': 'string'}, 'title': 'Caracteristicas', 'type': 'array'}, 'disponible': {'default': True, 'description': 'Si está disponible', 'title': 'Disponible', 'type': 'boolean'}, 'categoria': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default'

## Creando los agentes con structured Output

In [25]:
# Crear agentes con output estructurado

# Agente Extractor de Productos
agente_extractor_productos = LlmAgent(
    name="ExtractorProductos",
    model="gemini-2.5-flash",  # Pro maneja mejor output estructurado
    description="Extrae información estructurada de productos",
    output_schema=InformacionProducto,  # ¡Output estructurado!
    output_key='InformacionProducto',  # Clave para el output
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.1,
        max_output_tokens= 300
    ),
    instruction=(
        "Extrae información de productos del texto proporcionado."
        "Sigue exactamente el esquema definido."
        "Si no encuentras algún dato, usa None o lista vacía según corresponda."
        "Sé preciso con los precios y características."
    )
)

# Agente Analizador de Sentimientos
agente_sentimientos = LlmAgent(
    name="AnalizadorSentimientos",
    model=openai_model,
    description="Analiza el sentimiento y emociones en textos",
    output_schema=AnalisisSentimiento,  # ¡Output estructurado!
    output_key='AnalisisSentimiento',  # Clave para el output
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.3,
        max_output_tokens= 300
    ),
    instruction=(
        "Analiza el sentimiento del texto proporcionado."
        "Identifica emociones específicas presentes."
        "Lista aspectos positivos y negativos mencionados."
        "Asigna un nivel de confianza a tu análisis (0.0 a 1.0)."
    )
)

# Agente Extractor de Eventos
agente_eventos = LlmAgent(
    name="ExtractorEventos",
    model=anthropic_model,
    description="Extrae información de eventos de un texto",
    output_schema=ListaEventos,  # ¡Output estructurado con lista!
    output_key='ListaEventos',  # Clave para el output
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.2,
        max_output_tokens= 500
    ),
    instruction=(
        "Extrae TODOS los eventos mencionados en el texto."
        "Para cada evento, captura toda la información disponible."
        "Las fechas deben estar en formato YYYY-MM-DD."
        "Las horas en formato HH:MM."
        "Si hay múltiples eventos, inclúyelos todos en la lista."
    )
)

print("🎯 Agentes con output estructurado creados:")
print("   • ExtractorProductos → InformacionProducto")
print("   • AnalizadorSentimientos → AnalisisSentimiento")
print("   • ExtractorEventos → ListaEventos")


Invalid config for agent ExtractorProductos: output_schema cannot co-exist with agent transfer configurations. Setting disallow_transfer_to_parent=True, disallow_transfer_to_peers=True
Invalid config for agent AnalizadorSentimientos: output_schema cannot co-exist with agent transfer configurations. Setting disallow_transfer_to_parent=True, disallow_transfer_to_peers=True
Invalid config for agent ExtractorEventos: output_schema cannot co-exist with agent transfer configurations. Setting disallow_transfer_to_parent=True, disallow_transfer_to_peers=True


🎯 Agentes con output estructurado creados:
   • ExtractorProductos → InformacionProducto
   • AnalizadorSentimientos → AnalisisSentimiento
   • ExtractorEventos → ListaEventos


### Probando el primer agente de productos

In [28]:
APP_NAME = "output_1"
USER_ID = "user_7"
SESSION_ID = "session_007" # Using a fixed ID for simplicity

texto_producto = """
    El nuevo iPhone 15 Pro Max de Apple ya está disponible. 
    Con un precio de $1,199, incluye cámara de 48MP, pantalla de 6.7 pulgadas,
    chip A17 Pro y batería de larga duración. Disponible en titanio.
    """

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_productos = Runner(agent=agente_extractor_productos,app_name=APP_NAME,session_service=session_service)

await call_agent_async(texto_producto,
                        runner=runner_productos,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


>>> Consulta del usuario: 
    El nuevo iPhone 15 Pro Max de Apple ya está disponible. 
    Con un precio de $1,199, incluye cámara de 48MP, pantalla de 6.7 pulgadas,
    chip A17 Pro y batería de larga duración. Disponible en titanio.
    
<<< Respuesta del agente: {
  "nombre": "iPhone 15 Pro Max",
  "marca": "Apple",
  "precio": 1199,
  "caracteristicas": [
    "cámara de 48MP",
    "pantalla de 6.7 pulgadas",
    "chip A17 Pro",
    "batería de larga duración",
    "Disponible en titanio"
  ],
  "disponible": true,
  "categoria": null
}


### Probando el segundo agente de sentimientos

In [29]:
APP_NAME = "output_2"
USER_ID = "user_7"
SESSION_ID = "session_007" # Using a fixed ID for simplicity

texto_opinion = """
    ¡Me encanta mi nuevo laptop! La velocidad es increíble y la pantalla es hermosa.
    Sin embargo, la batería no dura tanto como esperaba y el precio fue algo alto.
    En general, estoy satisfecho con la compra.
    """

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_sentimientos = Runner(agent=agente_sentimientos,app_name=APP_NAME,session_service=session_service)

await call_agent_async(texto_producto,
                        runner=runner_sentimientos,
                        user_id=USER_ID,
                        session_id=SESSION_ID)



>>> Consulta del usuario: 
    El nuevo iPhone 15 Pro Max de Apple ya está disponible. 
    Con un precio de $1,199, incluye cámara de 48MP, pantalla de 6.7 pulgadas,
    chip A17 Pro y batería de larga duración. Disponible en titanio.
    
<<< Respuesta del agente: {"sentimiento":"neutral","confianza":0.9,"emociones":[],"aspectos_positivos":["cámara de 48MP","pantalla de 6.7 pulgadas","chip A17 Pro","batería de larga duración","disponible en titanio"],"aspectos_negativos":["precio de $1,199"]}


### Probando el tercer agente de eventos

In [30]:
APP_NAME = "output_3"
USER_ID = "user_7"
SESSION_ID = "session_007" # Using a fixed ID for simplicity

texto_eventos = """
    Recordatorio: La conferencia de IA será el 15 de marzo de 2024 a las 10:00 AM
    en el Centro de Convenciones. Participarán el Dr. Smith y la Dra. Johnson.
    Después, habrá un taller práctico a las 2:00 PM en el mismo lugar.
    """

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_eventos = Runner(agent=agente_eventos,app_name=APP_NAME,session_service=session_service)

await call_agent_async(texto_eventos,
                        runner=runner_eventos,
                        user_id=USER_ID,
                        session_id=SESSION_ID)



>>> Consulta del usuario: 
    Recordatorio: La conferencia de IA será el 15 de marzo de 2024 a las 10:00 AM
    en el Centro de Convenciones. Participarán el Dr. Smith y la Dra. Johnson.
    Después, habrá un taller práctico a las 2:00 PM en el mismo lugar.
    
<<< Respuesta del agente: {"eventos": [{"titulo": "Conferencia de IA", "fecha": "2024-03-15", "hora": "10:00", "ubicacion": "Centro de Convenciones", "participantes": ["Dr. Smith", "Dra. Johnson"], "descripcion": null}, {"titulo": "Taller pr\u00e1ctico", "fecha": "2024-03-15", "hora": "14:00", "ubicacion": "Centro de Convenciones", "participantes": [], "descripcion": null}], "total_eventos": 2}


____

## 🎓 RESUMEN DEL TUTORIAL 2: CONTROL DE LLMs
---

### 1️⃣ LiteLLM - Flexibilidad de Modelos
- ✅ Usa cualquier modelo con el prefijo `litellm/`
- ✅ Soporte para **100+ modelos diferentes**
- ✅ Cambio fácil entre proveedores
- ✅ Optimización de **costos y rendimiento**

---

### 2️⃣ Parámetros de LLM - Control
- ✅ `temperature`: controla la **creatividad vs consistencia**
- ✅ `max_tokens`: limita la **longitud** de la respuesta y ayuda a controlar costos
- ✅ `top_p`: alternativa a `temperature` para muestreo
- ✅ Configuración ajustada según el **caso de uso**

---

### 3️⃣ Output Estructurado - Respuestas Predecibles
- ✅ Uso de **Pydantic** para definir esquemas de respuesta
- ✅ **Validación automática** de los datos generados por el modelo
- ✅ Integración sencilla con otros sistemas
- ✅ Producción de **JSON consistente y tipado**

---


---

## 🎉 ¡Felicitaciones!

Has completado el Tutorial 2 de Google ADK. Ahora tienes el poder de:
- Integrar cualquier LLM en tus agentes
- Controlar finamente su comportamiento
- Obtener respuestas estructuradas y validadas

**Siguiente paso**: Tutorial 3 - Herramientas  🛠️

---

**¿Preguntas?** Déjalas en los comentarios del video o consulta la documentación oficial.

**¡Feliz codificación con ADK!** 🚀

## 📚 Recursos Adicionales

### Enlaces Útiles

- **Documentación ADK**: [https://google.github.io/adk-docs/](https://google.github.io/adk-docs/)
- **LiteLLM Docs**: [https://docs.litellm.ai/](https://docs.litellm.ai/)
- **Pydantic Docs**: [https://docs.pydantic.dev/](https://docs.pydantic.dev/)
- **Modelos Soportados**: [Lista completa de LiteLLM](https://docs.litellm.ai/docs/providers)
